#### Libraries

In [7]:
import geopandas as gpd
from datetime import timedelta
import openeo
import rasterio
from rasterio.features import rasterize
import rioxarray
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from rasterio.crs import CRS
import fiona
from rasterio.mask import mask

from openeo.processes import if_, is_nan

from files.utils_BAP import (calculate_cloud_mask, calculate_cloud_coverage_score,
                           calculate_date_score, calculate_distance_to_cloud_score,
                           calculate_distance_to_cloud_score, aggregate_BAP_scores,
                           create_rank_mask)


#### Download Imagery as NETCDF

In [ ]:
gdf = gpd.read_file(r"files\sample_geojson")
gdf = gdf.to_crs(epsg=4326)

# extract date and shift the date to 2 weeks before the harvest time
lowest_date = gdf.loc[0, 'HDate'] - timedelta(days=14)
highest_date = gdf.iloc[-1]['HDate'] - timedelta(days=14)

# make the date in Years, Month and date
highest_date= highest_date.strftime('%Y-%m-%d')
lowest_date = lowest_date.strftime('%Y-%m-%d')

# delete date columns or else it won't serialise to JSON
del gdf['SDate']
del gdf['HDate']

# convert to JSON
area = eval(gdf.to_json())

In [ ]:
max_cloud_cover = 70
spatial_resolution = 20
temporal_extent = ['2023-08-01', '2023-08-31']

In [ ]:
connection =openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

scl = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=temporal_extent,
    bands=["SCL"],
    max_cloud_cover=max_cloud_cover,
    spatial_extent={
                        "west": 78.2069150142717859, 
                        "south": 19.3370105417870697,
                        "east": 78.9373164563397438, 
                        "north": 19.7840957974622071, 
                        "crs": 'EPSG:4326'
                    }
).resample_spatial(spatial_resolution)

scl = scl.apply(lambda x: if_(is_nan(x), 0, x))

In [ ]:
cloud_mask =  calculate_cloud_mask(scl)

coverage_score = calculate_cloud_coverage_score(cloud_mask, area, scl)

date_score = calculate_date_score(scl)

dtc_score = calculate_distance_to_cloud_score(cloud_mask, spatial_resolution)

weights = [1, 0.8, 0.5]
score = aggregate_BAP_scores(dtc_score, date_score, coverage_score, weights)
score = score.mask(scl.band("SCL") == 0)

rank_mask = create_rank_mask(score)

In [ ]:
rgb_bands = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent = temporal_extent,
    bands = ["B02", "B03","B04"],
    max_cloud_cover=max_cloud_cover,
    spatial_extent={
                        "west": 78.2069150142717859, 
                        "south": 19.3370105417870697,
                        "east": 78.9373164563397438, 
                        "north": 19.7840957974622071, 
                        "crs": 'EPSG:4326'
                    }
)

composite = rgb_bands.mask(rank_mask).mask(cloud_mask).aggregate_temporal_period("month","first")

In [ ]:
job = composite.execute_batch(
    title="BAP Composite",
    out_format="netcdf"
)
results = job.get_results()
results.download_files(r'files/')

#### Export Downloaded NETCDF Imagery as Tiff imagery

In [ ]:
# load in the nz file and use the polygons in 
satellite_image_path = r"files\openEO.nc"


composite_ds = rioxarray.open_rasterio(satellite_image_path, decode_times=False)

# check dimension 
print(f'Dimension before reduction: {composite_ds.dims}')

composite_ds = composite_ds.squeeze(axis=0)

# check dimension 
print(f'Dimension after reduction: {composite_ds.dims}')

# export the netcdf to tif raster
composite_ds.rio.to_raster(r'files/all_bands.tif')

#### Single Channel Mask

In [8]:
gdf = gpd.read_file(r"files\sample_geojson")

# convert geojson to shapefile
# gdf.to_file(r'files/sample_shapefile.shp')
# gdf = gdf.to_crs(epsg=4326)

satellite_image_path = r"files\all_bands.tif"
with rasterio.open(satellite_image_path, 'r+') as src:
    satellite_meta = src.meta
    satellite_transform = src.transform
    satellite_shape = (src.height, src.width) 

    # change geojson crs to the crs of the image
    image_crs  = 'EPSG:32644'
    gdf = gdf.to_crs(image_crs)

    # extract polygons
    shapes = [feature["geometry"] for feature in gdf.iterfeatures()]

    # Filter NoneType from building shape
    shapes = list(filter(None, shapes))

    # Clip the raster with Polygon
    out_img, out_transform = mask(src, shapes, crop=False)

    # since there are some values that are lesser than zero, these are nan values, turn them to zero
    out_img_updated= np.where(out_img<=0, 0, out_img)

    # make sure that values that are not zero turns out to be 255  (these are the masks)
    out_img_unique = np.where(out_img_updated!= 0, 255, out_img_updated)

In [9]:
# since we are working with a single label, the mask should have one channel
mask = out_img_unique[0, :, :]

# make sure the dimension of the image is (channel, Height, Width) and not (Height, Width)
# this needs to be done, else the mask would wrongly be calculated
expand_mask = np.expand_dims(mask, axis=0)

In [ ]:
# export mask
out_tif = r'files/mask_single_channel.tiff'
out_meta= {'driver': 'GTiff',
                 'dtype': 'uint8',
                 'height': expand_mask.shape[1],
                 'width': expand_mask.shape[2],
                 'transform': out_transform,
                 'count': 1,
                 'crs': src.crs 
}
with rasterio.open(out_tif, "w", **out_meta) as dest:
        dest.write(expand_mask.astype('uint8'))
